# Building Dataset. Part 1
### By **Néstor Suat** in 2020

**Descripción:** Contruyendo el dataset Parte 1. Generar los archivos para BRAT Annotation Tools.

**Input:**
* TSV con los tweets seleccionados

**Output:**
* Files *.txt and *.ann
***

### Importando librerías

Como estamos en un archivo afuera se necesita agregar la dirección ../ (raíz del proyexto) para importar la librería de preprocesamiento.

In [1]:
import pandas as pd
import spacy  # For preprocessing
import re

import sys
sys.path.insert(0, '../../../')

from classes.wordsegmentation import WordSegmentation

In [2]:
nlp = spacy.load("es_core_news_lg") # disabling Named Entity Recognition for speed

### Importando datasets

In [3]:
file = 'ner_dataset_shuffle.tsv'
dir_ = "../../../data/v1/NER/"

In [44]:
dataset = pd.read_csv(dir_+file, delimiter = "\t", quoting = 3)
del dataset['Unnamed: 0']
dataset = dataset['text']
dataset[0]

'#ElNoctámbulo Aparatoso accidente en la Avenida Suba con calle 127. El conductor se salvó de milagro. #ArribaBogotá https://t.co/D6qLLsufhG'

In [45]:
#Seleccionando 50
df = dataset.sample(n=50)

### Aplicando limpieza

In [46]:
seg = WordSegmentation(dir_+'spanish_count_1w_small_v2_twitter.txt')

In [47]:
def word_segmentation(pattern, text):
    search = re.search(pattern,text)
    while search:    
        s = search.start()
        e = search.end()                
        text = text[:s] + ' ' + ' '.join(seg.segment(text[s+1:e])) +' '+ text[e:]        
        search = re.search(pattern,text)        
    return text

def clean_fn(doc):    
    text = doc
    pattern = "(@[A-Za-z0-9äÄëËïÏöÖüÜáéíóúáéíóúÁÉÍÓÚÂÊÎÔÛâêîôûàèìòùÀÈÌÒÙñÑ_]+)"
    text = word_segmentation(pattern,text)
    pattern = "(#[A-Za-z0-9äÄëËïÏöÖüÜáéíóúáéíóúÁÉÍÓÚÂÊÎÔÛâêîôûàèìòùÀÈÌÒÙñÑ_]+)"
    text = word_segmentation(pattern,text)
    return text
    
def preText(text):        
    pre = re.sub("&[A-Za-z]+;", ' ', text) #Eliminar códigos ASCII
    pre = re.sub("(\w+:\/\/\S+)",' ',pre) #Eliminar links http y https
    pre = re.sub("([^A-Za-z0-9äÄëËïÏöÖüÜáéíóúáéíóúÁÉÍÓÚÂÊÎÔÛâêîôûàèìòùÀÈÌÒÙñÑ,;.:*\-\[\]¿?¡!\"\"()_'/])",' ',pre) #Eliminar caracteres especiales como emoticones, exceptuando los signos de puntuación y tildes.
    pre = re.sub(r'([;.:\-\[\]¿?¡!#\"()]){3,}',r'\1\1',pre) #Si repite un caracters especial más de 3 veces ej. """"
    pre = re.sub(r'([a-zA-Z])\1{2,}',r'\1\1',pre) #Si repite una letra más de dos veces las reduce a dos repeticiones goool => gool        
    pre = re.sub(r'(\s){2,}',r' ',pre) #Eliminar espacios seguidos              
    return pre.strip()

In [48]:
brief_cleaning = (clean_fn(str(row)) for row in df)
txt = [preText(doc.text) for doc in nlp.pipe(brief_cleaning, batch_size=50, n_threads=4)]

In [42]:
txt

['bogotatransito accidente de varios sentido sur-norte en la autonorte a la altura de la 116.',
 'Más imprudentes en vía por pasarse el semáforo en rojo par de ciclistas colisionaron sin gravedad sobre la 98 con 11 gente más educación sobre una bicicleta',
 'natalia medina gc bogotatransito transmilenio sectormovilidad Hay un accidente en la av suba con cra 91 que involucra a un sitp..desvían particulares por el carril exclusivo de transmilenio',
 'CRA 69 X CL 21 : Se presenta incidente vial entre servicio especial y furgón',
 'bogotatransito Cra 9 con Cll 116 sobre el puente reportan accidente',
 'Accidente en la 106 con 9',
 'transito policia sectormovilidad Choque simple de zonal(sin datos) vs particular en la Carrera 11 este con calle 47 C Sur, no hay paso se requiere tránsito , la novedad está desde las 16:25',
 'sectormovilidad transmilenio accidentado en Av. Suba, a la altura de Av. Calle 139 con carrera 90.',
 'noticias bta ¡Hola! A la altura de la carrera 94 sentido oriente oc

In [49]:
new = pd.DataFrame(df)
new['pre'] = txt
new.to_csv('test.tsv',sep='\t')

### Generando archivos

In [72]:
#Generando txt y ann
i=1
for tweet in txt:
    name = dir_+'brat/'+'tweet-'+str(i)+'.txt'
    #print(name)  
    with open(name, "w") as file:
        file.write(tweet)
    name = dir_+'brat/'+'tweet-'+str(i)+'.ann'
    with open(name, "w") as file:
        file.write('')
    i+=1